# Amazon SageMaker를 활용한 신용 위험 예측, 설명 가능성 및 편향 감지

![Credit risk explainability use case](credit_risk_prediction.png)

1. [개요](#Overview)
1. [사전 요구 사항 및 데이터](#Prerequisites-and-Data)
    1. [SageMaker 초기화](#Initialize-SageMaker)
    1. [데이터 다운로드](#Download-data)
    1. [데이터 로딩: 독일 신용(업데이트) 데이터셋](#Loading-the-data:-German-credit-Dataset) 
    1. [데이터 검사](#Data-inspection) 
    1. [데이터 전처리 모델 및 S3 업로드](#Preprocess-and-Upload-Training-Data) 
1. [XGBoost 모델 훈련](#Train-XGBoost-Model)
    1. [모델 훈련](#Train-Model)
1. [추론 파이프라인이 포함된 SageMaker 모델 생성](#Create-SageMaker-Model)
1. [Amazon SageMaker Clarify](#Amazon-SageMaker-Clarify)
    1. [예측 설명](#Explaining-Predictions)
        1. [설명 가능성 보고서 보기](#Viewing-the-Explainability-Report)
        2. [개별 불량 신용 예측 예시 설명](#Explaining-individual-prediction)
    2. [편향 이해하기](#Bias-Detection)
        1. [훈련 전 편향 지표](#pre-training)
        2. [훈련 후 편향 지표](#post-training)
1. [정리](#Clean-Up)
1. [추가 리소스](#Additional-Resources)

## 1. 개요
Amazon SageMaker는 데이터 과학자와 개발자가 ML을 위해 특별히 제작된 광범위한 기능을 통합하여 고품질 머신 러닝(ML) 모델을 빠르게 준비, 구축, 훈련 및 배포할 수 있도록 지원합니다.

[Amazon SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/)는 잠재적 편향을 감지하고 이러한 모델이 예측을 수행하는 방법을 설명하는 데 도움을 줌으로써 머신 러닝 모델을 개선하는 데 도움이 됩니다. SageMaker Clarify가 제공하는 공정성 및 설명 가능성 기능은 AWS 고객이 신뢰할 수 있고 이해하기 쉬운 머신 러닝 모델을 구축할 수 있도록 하는 단계를 제공합니다.

Amazon SageMaker는 Scikit-Learn, XGBoost, TensorFlow, PyTorch, MXNet 또는 Chainer와 같은 지원되는 프레임워크를 위한 머신 러닝 및 딥 러닝 프레임워크용 사전 제작된 이미지를 제공합니다. 이러한 이미지에는 해당 프레임워크와 Pandas 및 NumPy와 같은 추가 Python 패키지가 미리 로드되어 있어 모델 훈련을 위한 자체 코드를 작성할 수 있습니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/algorithms-choose.html#supported-frameworks-benefits)를 참조하세요.

[Amazon SageMaker Studio](https://aws.amazon.com/sagemaker/studio/)는 노트북, 실험 관리, 자동 모델 생성, 디버깅, 모델 및 데이터 드리프트 감지를 포함한 모든 ML 개발 활동을 수행할 수 있는 단일 웹 기반 시각적 인터페이스를 제공합니다.

이 SageMaker Studio 노트북에서는 SageMaker를 사용하여 모델을 훈련하고, 배포 가능한 SageMaker 모델을 생성하며, 편향 감지 및 설명 가능성을 제공하여 데이터를 분석하고 모델의 예측 결과를 이해하는 방법을 강조합니다.
이 샘플 노트북은 다음 과정을 안내합니다:

1. 신용 위험 데이터셋 다운로드 및 탐색 - [남부 독일 신용(업데이트) 데이터셋](https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29)
2. sklearn을 사용한 데이터셋 전처리
3. XGBoost를 사용한 데이터셋에 GBM 모델 훈련
4. 입력 데이터를 전처리하고 인스턴스별 예측 결과를 생성하기 위한 추론 파이프라인 모델(sklearn 모델과 XGBoost 모델 함께) 구축
5. 단일 모델 호스팅 및 점수 매기기(선택 사항)
6. 훈련 및 테스트 데이터셋에 대한 SageMaker 모델의 Kernel SHAP 값을 제공하는 SageMaker Clarify 작업
7. 데이터에 대한 훈련 전 편향 지표를 포함한 편향 지표를 제공하는 SageMaker Clarify 작업

![신용 위험 설명 가능성 모델 추론](clarify_inf_pipeline_arch.jpg)

## 2. 사전 준비 및 데이터 탐색과 특성 공학
### SageMaker 초기화

In [ ]:
# cell 01
from io import StringIO
import os
import time
import sys
import IPython
from time import gmtime, strftime

import boto3
import numpy as np
import pandas as pd
import urllib

import sagemaker
from sagemaker.s3 import S3Uploader
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost import XGBoost
from sagemaker.s3 import S3Downloader
from sagemaker.s3 import S3Uploader
from sagemaker import Session
from sagemaker import get_execution_role
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn import SKLearnModel
from sagemaker.pipeline import PipelineModel


session = Session()
bucket = session.default_bucket()
prefix = "sagemaker/sagemaker-clarify-credit-risk-model"
region = session.boto_region_name

# Define IAM role
role = get_execution_role()

### 데이터 다운로드

먼저, 데이터를 __다운로드__하고 `data` 폴더에 저장하세요.


$^{[2]}$ Ulrike Grömping
Beuth University of Applied Sciences Berlin
연락처 정보가 있는 웹사이트: https://prof.beuth-hochschule.de/groemping/.

In [ ]:
# cell 02
S3Downloader.download(
    "s3://sagemaker-sample-files/datasets/tabular/uci_statlog_german_credit_data/SouthGermanCredit.asc",
    "data",
)

In [ ]:
# cell 03
credit_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
    "credit_risk",
]

$`laufkont = status`
                                               
 1 : 당좌 계좌 없음                       
 2 : ... < 0 DM                                
 3 : 0<= ... < 200 DM                          
 4 : ... >= 200 DM / 최소 1년 이상의 급여

$`laufzeit = duration`
     

$`moral = credit_history`
                                                
 0 : 과거 상환 지연                            
 1 : 중요 계좌/다른 곳에서의 신용 대출   
 2 : 대출 이력 없음/모든 대출 정상 상환
 3 : 현재까지 기존 대출 정상 상환   
 4 : 이 은행의 모든 대출 정상 상환    

$`verw = purpose`
                        
 0 : 기타             
 1 : 자동차(신차)          
 2 : 자동차(중고차)         
 3 : 가구/장비
 4 : 라디오/텔레비전   
 5 : 가전제품
 6 : 수리            
 7 : 교육          
 8 : 휴가           
 9 : 재교육         
 10 : 사업          

$`hoehe = amount`
     

$`sparkont = savings`
                               
 1 : 알 수 없음/저축 계좌 없음
 2 : ... <  100 DM             
 3 : 100 <= ... <  500 DM      
 4 : 500 <= ... < 1000 DM      
 5 : ... >= 1000 DM            

$`beszeit = employment_duration`
                     
 1 : 실업 상태      
 2 : < 1년          
 3 : 1 <= ... < 4년
 4 : 4 <= ... < 7년
 5 : >= 7년        

$`rate = installment_rate`
                   
 1 : >= 35         
 2 : 25 <= ... < 35
 3 : 20 <= ... < 25
 4 : < 20          

$`famges = personal_status_sex`
                                         
 1 : 남성 : 이혼/별거           
 2 : 여성 : 비독신 또는 남성 : 독신
 3 : 남성 : 기혼/사별              
 4 : 여성 : 독신                     

$`buerge = other_debtors`
                 
 1 : 없음        
 2 : 공동 신청인
 3 : 보증인   

$`wohnzeit = present_residence`
                     
 1 : < 1년          
 2 : 1 <= ... < 4년
 3 : 4 <= ... < 7년
 4 : >= 7년        

$`verm = property`
                                              
 1 : 알 수 없음 / 재산 없음                    
 2 : 자동차 또는 기타                             
 3 : 건축 저축 계약/생명 보험
 4 : 부동산                              

$`alter = age`
     

$`weitkred = other_installment_plans`
           
 1 : 은행  
 2 : 상점
 3 : 없음  

$`wohn = housing`
             
 1 : 무상
 2 : 임대    
 3 : 자가     

$`bishkred = number_credits`
         
 1 : 1   
 2 : 2-3 
 3 : 4-5 
 4 : >= 6

$`beruf = job`
                                               
 1 : 실업자/비숙련 - 비거주자       
 2 : 비숙련 - 거주자                      
 3 : 숙련된 직원/공무원                 
 4 : 관리자/자영업자/고도로 자격을 갖춘 직원

$`pers = people_liable`
              
 1 : 3명 이상
 2 : 0~2명   

$`telef = telephone`
                              
 1 : 아니오                       
 2 : 예(고객 이름으로)

$`gastarb = foreign_worker`
        
 1 : 예
 2 : 아니오 

$`kredit = credit_risk`
         
 0 : 나쁨 
 1 : 좋음

### 데이터 검사

In [ ]:
# cell 04
training_data = pd.read_csv(
    "data/SouthGermanCredit.asc",
    names=credit_columns,
    header=0,
    sep=r" ",
    engine="python",
    na_values="?",
).dropna()

print(training_data.head())

다양한 특성의 분포에 대한 히스토그램을 그리는 것은 데이터를 시각화하는 좋은 방법입니다.

In [ ]:
# cell 05
%matplotlib inline
training_data["credit_risk"].value_counts().sort_values().plot(
    kind="bar", title="Counts of Target", rot=0
)

### 원시 훈련 및 테스트 CSV 파일 생성하기

Korean: 원시 훈련 및 테스트 CSV 파일 생성하기

In [ ]:
# cell 06
# prepare raw test data
test_data = training_data.sample(frac=0.1)
test_data = test_data.drop(["credit_risk"], axis=1)
test_filename = "test.csv"
test_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
]
test_data.to_csv(test_filename, index=False, header=True, columns=test_columns, sep=",")

# prepare raw training data
train_filename = "train.csv"
training_data.to_csv(train_filename, index=False, header=True, columns=credit_columns, sep=",")

### 데이터 인코딩 및 업로드
여기서는 훈련 및 테스트 데이터를 인코딩합니다. 입력 데이터 인코딩은 SageMaker Clarify에는 필요하지 않지만, XGBoost 모델에는 필요합니다.

In [ ]:
# cell 07
test_raw = S3Uploader.upload(test_filename, "s3://{}/{}/data/test".format(bucket, prefix))
print(test_raw)

In [ ]:
# cell 08
train_raw = S3Uploader.upload(train_filename, "s3://{}/{}/data/train".format(bucket, prefix))
print(train_raw)

### SageMaker Processing 작업을 통한 전처리 및 특성 엔지니어링

SageMaker Processing 작업을 사용하여 원시 데이터에 대한 전처리를 수행하겠습니다. SageMaker Processing은 여기서 사용할 SKlearn을 위한 사전 구축된 컨테이너를 제공합니다. 추론 요청 전처리에 사용할 수 있는 sklearn 모델을 출력하겠습니다.

In [ ]:
# cell 09
sklearn_processor = SKLearnProcessor(
    role=role,
    base_job_name="sagemaker-clarify-credit-risk-processing-job",
    instance_type="ml.m5.large",
    instance_count=1,
    framework_version="0.20.0",
)

프로세싱 작업에서 실행할 전처리 스크립트를 살펴볼 수 있습니다.

In [ ]:
# cell 10
!pygmentize processing/preprocessor.py

#### 참고: 이 셀은 약 5-8분 동안 실행됩니다! 기다려 주세요.
SageMaker Processing에 대한 추가 문서는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)에서 확인할 수 있습니다.

In [ ]:
# cell 11
raw_data_path = "s3://{0}/{1}/data/train/".format(bucket, prefix)
train_data_path = "s3://{0}/{1}/data/preprocessed/train/".format(bucket, prefix)
val_data_path = "s3://{0}/{1}/data/preprocessed/val/".format(bucket, prefix)
model_path = "s3://{0}/{1}/sklearn/".format(bucket, prefix)


sklearn_processor.run(
    code="processing/preprocessor.py",
    inputs=[
        ProcessingInput(
            input_name="raw_data", source=raw_data_path, destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data", source="/opt/ml/processing/train", destination=train_data_path
        ),
        ProcessingOutput(
            output_name="val_data", source="/opt/ml/processing/val", destination=val_data_path
        ),
        ProcessingOutput(
            output_name="model", source="/opt/ml/processing/model", destination=model_path
        ),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
    logs=False,
)

## XGBoost 모델 훈련
이 단계에서는 전처리된 데이터로 XGBoost 모델을 훈련시키겠습니다. SageMaker에서 제공하는 내장 XGBoost 컨테이너와 함께 자체 훈련 스크립트를 사용할 것입니다.

또는 자체 사용 사례의 경우, SageMaker Clarify로 처리하기 위해 (다른 곳에서 훈련된) 자체 모델을 SageMaker로 가져올 수도 있습니다.

In [ ]:
# cell 12
!pygmentize training/train_xgboost.py

### XGBoost Estimator 설정하기

다음으로 설정해 보겠습니다:
1. XGBoost 알고리즘을 위한 하이퍼파라미터의 사전 정의된 값
1. SageMaker용 XGBoost Estimator

In [ ]:
# cell 13
hyperparameters = {
    "max_depth": "5",
    "eta": "0.1",
    "gamma": "4",
    "min_child_weight": "6",
    "silent": "1",
    "objective": "binary:logistic",
    "num_round": "100",
    "subsample": "0.8",
    "eval_metric": "auc",
    "early_stopping_rounds": "20",
}

entry_point = "train_xgboost.py"
source_dir = "training/"
output_path = "s3://{0}/{1}/{2}".format(bucket, prefix, "xgb_model")
code_location = "s3://{0}/{1}/code".format(bucket, prefix)

estimator = XGBoost(
    entry_point=entry_point,
    source_dir=source_dir,
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    instance_type="ml.c5.xlarge",
    instance_count=1,
    framework_version="0.90-2",
    py_version="py3",
    role=role,
)

### SageMaker 훈련

이제 모델을 훈련시킬 시간입니다

#### 참고: 이 셀은 약 5-8분 동안 실행됩니다! 기다려 주세요.
SageMaker 훈련에 대한 자세한 문서는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/train-model.html)에서 확인할 수 있습니다.

In [ ]:
# cell 14
job_name = f"credit-risk-xgb-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

train_input = TrainingInput(
    "s3://{0}/{1}/data/preprocessed/train/".format(bucket, prefix), content_type="csv"
)
val_input = TrainingInput(
    "s3://{0}/{1}/data/preprocessed/val/".format(bucket, prefix), content_type="csv"
)

inputs = {"train": train_input, "validation": val_input}

estimator.fit(inputs, job_name=job_name)

## 4. SageMaker 모델 생성

엔드포인트로 배포하거나 SageMaker Clarify와 함께 사용할 수 있는 SageMaker 추론 파이프라인 모델을 준비하겠습니다:
  1. 원시 데이터를 입력으로 받음
  1. 앞서 구축한 SKlearn 모델로 데이터 전처리
  1. SKlearn 모델의 출력을 자동으로 XGBoost 모델의 입력으로 전달
  1. XGBoost 모델에서 최종 추론 결과 제공
  

추론 파이프라인에 대한 자세한 내용은 다음 문서를 참조하세요: https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html

### 모델 아티팩트 검색하기

먼저, 두 개의 Amazon SageMaker Model 객체를 생성해야 합니다. 이 객체들은 훈련의 아티팩트(Amazon S3에 저장된 직렬화된 모델 아티팩트)를 추론에 사용되는 Docker 컨테이너와 연결합니다. 이를 위해 Amazon S3에 있는 직렬화된 모델의 경로를 가져와야 합니다. 여기서 SKlearn 및 XGBoost 모델의 모델 데이터 위치를 정의합니다.

In [ ]:
# cell 15
preprocessor_model_data = "s3://{}/{}/{}".format(bucket, prefix, "sklearn") + "/model.tar.gz"

xgboost_model_data = (
    "s3://{}/{}/{}/{}".format(bucket, prefix, "xgb_model", job_name) + "/output/model.tar.gz"
)

### SageMaker SKlearn 모델 객체 생성하기

다음 단계는 다음과 같은 중요한 정보를 포함하는 `SKlearnModel` 객체를 생성하는 것입니다:
  1. sklearn 모델 데이터의 위치
  1. 사용자 정의 추론 코드
  1. 사용할 SKlearn 버전(전처리 중에 사용한 버전과 동일한지 확인)

이 모델을 호스팅하기 위해 입력과 출력을 처리하고 변환을 실행하는 데 사용되는 사용자 정의 추론 스크립트를 제공합니다.

추론 스크립트는 `inference/sklearn/inference.py` 파일에 구현되어 있습니다. 이 사용자 정의 스크립트는 다음을 정의합니다:

- 추론 요청을 전처리하기 위한 사용자 정의 `input_fn`. 우리의 입력 함수는 CSV 입력만 허용하며, 입력을 Pandas 데이터프레임으로 로드하고 특성 열 이름을 데이터프레임에 할당합니다.
- 입력에 대해 변환을 실행하기 위한 사용자 정의 `predict_fn`
- 모델을 역직렬화하기 위한 사용자 정의 `model_fn`

SageMaker SKlearn 컨테이너에서 제공하는 `output_function`의 기본 구현을 사용할 것입니다. 자세한 내용은 다음을 확인하세요: https://github.com/aws/sagemaker-scikit-learn-container

In [ ]:
# cell 16
!pygmentize inference/sklearn/inference.py

이제 SKLearnModel 객체를 정의해보겠습니다.

Korean: 이제 SKLearnModel 객체를 정의해보겠습니다.

In [ ]:
# cell 17
sklearn_inference_code_location = "s3://{}/{}/{}/code".format(bucket, prefix, "sklearn")

sklearn_model = SKLearnModel(
    name="sklearn-model-{0}".format(str(int(time.time()))),
    model_data=preprocessor_model_data,
    entry_point="inference.py",
    source_dir="inference/sklearn/",
    code_location=sklearn_inference_code_location,
    role=role,
    sagemaker_session=session,
    framework_version="0.20.0",
    py_version="py3",
)

### SageMaker XGBoost 모델 객체 생성하기

이전 단계와 유사하게, XGBoost 모델 객체를 생성할 수 있습니다. 여기서도 사용자 정의 추론 스크립트를 제공해야 합니다.

추론 스크립트는 `inference/xgboost/inference.py` 파일에 구현되어 있습니다. 이 사용자 정의 스크립트는 다음을 정의합니다:

- 추론 요청을 전처리하기 위한 사용자 정의 `input_fn`. 이 입력 함수는 JSON 요청뿐만 아니라 기본 XGBoost 컨테이너가 지원하는 모든 콘텐츠 유형을 처리할 수 있습니다. 추가 정보는 다음 링크를 참조하세요: https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/encoder.py. JSON 콘텐츠 유형을 추가한 이유는 추론 파이프라인에서 컨테이너 간 기본 요청 콘텐츠 유형이 JSON이기 때문입니다.

- 모델 역직렬화를 위한 사용자 정의 `model_fn`

추론 스크립트를 살펴보겠습니다.

In [ ]:
# cell 18
!pygmentize inference/xgboost/inference.py

이제 XGBoost 모델 객체를 정의해 보겠습니다.

In [ ]:
# cell 19
xgboost_inference_code_location = "s3://{}/{}/{}/code".format(bucket, prefix, "xgb_model")

xgboost_model = XGBoostModel(
    name="xgb-model-{0}".format(str(int(time.time()))),
    model_data=xgboost_model_data,
    entry_point="inference.py",
    source_dir="inference/xgboost/",
    code_location=xgboost_inference_code_location,
    framework_version="0.90-2",
    py_version="py3",
    role=role,
    sagemaker_session=session,
)

## SageMaker Pipeline Model 객체 생성하기

모델이 준비되면 PipelineModel 객체를 구축하고 deploy() 메서드를 호출하여 파이프라인에 배포할 수 있습니다.

In [ ]:
# cell 20
pipeline_model_name = "credit-risk-inference-pipeline-{0}".format(str(int(time.time())))

pipeline_model = PipelineModel(
    name=pipeline_model_name,
    role=role,
    models=[sklearn_model, xgboost_model],
    sagemaker_session=session,
)

### 설명 가능성 작업을 설정할 때 필요하므로 `model name`을 기록해 두세요.

In [ ]:
# cell 21
pipeline_model.name

### 모델 배포 (선택 사항 - Clarify에는 필요하지 않음)

모델을 배포하고 추론 파이프라인을 테스트해보겠습니다.

#### 참고: 이 셀은 약 5-8분 동안 실행됩니다! 기다려 주세요.
SageMaker 추론에 대한 자세한 문서는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)에서 확인할 수 있습니다.

In [ ]:
# cell 22
endpoint_name = "credit-risk-pipeline-endpoint-{0}".format(str(int(time.time())))
print(endpoint_name)

pipeline_model.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name
)

### 추론 (선택 사항 - Clarify에는 필요하지 않음)

이제 모델이 배포되었으니, 선택적으로 이 노트북에서 앞서 생성한 원시 테스트 데이터로 테스트해보겠습니다.

In [ ]:
# cell 23
test_dataset = S3Downloader.read_file(test_raw)

predictor = sagemaker.predictor.Predictor(
    endpoint_name,
    session,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

predictions = predictor.predict(test_dataset)

In [ ]:
# cell 24
predictions

## 5. Amazon SageMaker Clarify

사전 요구사항:

1. 엔드포인트에 배포할 수 있는 [SageMaker Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)

2. 입력 데이터셋
                  
3. SHAP 기준선

이제 모델 설정이 완료되었습니다. SageMaker Clarify를 시작해 보겠습니다!

In [ ]:
# cell 25
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.c4.xlarge", sagemaker_session=session
)

### 예측 설명하기
모델이 내린 결정의 _이유_에 대한 설명을 요구하는 비즈니스 요구사항과 법적 규제가 증가하고 있습니다. SageMaker Clarify는 [SHAP 라이브러리](https://github.com/slundberg/shap)를 사용하여 각 입력 특성이 최종 결정에 기여하는 정도를 설명합니다. SageMaker Clarify는 확장 가능하고 효율적인 [Kernel SHAP](https://github.com/slundberg/shap#model-agnostic-example-with-kernelexplainer-explains-any-function) 구현을 사용하며, 여러 처리 인스턴스로 Spark 기반 병렬 처리를 선택할 수 있습니다. Kernel SHAP 및 SageMaker Clarify는 모델에 구애받지 않는 특성 기여도 접근 방식을 가지고 있습니다. [SageMaker 모델](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)로 표현되는 모든 머신 러닝 모델은 설명 가능성을 위해 Clarify와 함께 사용할 수 있습니다.

Clarify와 SHAP를 사용한 설명 가능성에 대한 자세한 정보는 다음과 같습니다:

    https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html
    
    https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-shapley-values.html
    
    https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf

### SHAP을 위한 기준선 생성하기

Korean: 기준선 생성하기

머신 러닝 모델의 예측을 해석하는 데 있어 SHAP(SHapley Additive exPlanations)는 매우 유용한 도구입니다. SHAP 값은 각 특성이 모델의 예측에 얼마나 기여하는지를 보여줍니다.

대조적 설명 기법으로서, SHAP 값은 기준 샘플에 대해 생성된 합성 데이터에 모델을 평가하여 계산됩니다. 동일한 사례에 대한 설명은 이 기준 샘플의 선택에 따라 달라질 수 있습니다.

우리는 불량 신용 예측을 설명하는 데 관심이 있습니다. 따라서, 기준 선택이 E(x)를 1(우량 신용 클래스에 속함)에 더 가깝게 하고자 합니다.

기준을 만들기 위해 [최빈값](https://en.wikipedia.org/wiki/Mode_(statistics)) 통계를 사용합니다. 최빈값은 범주형 변수에 좋은 선택입니다. 기준에 대한 모델 예측이 우량 신용 클래스에 대해 높은 확률을 가지므로 기준에 대한 요구 사항을 충족한다는 것을 관찰할 수 있습니다.

유익한 기준과 비유익한 기준 선택에 대한 자세한 정보는 [설명 가능성을 위한 SHAP 기준](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)을 참조하세요.

In [ ]:
# cell 26
# load the raw training data in a data frame
raw_train_df = pd.read_csv("train.csv", header=0, names=None, sep=",")

# drop the target column
baseline = raw_train_df.drop(["credit_risk"], axis=1).mode().iloc[0].values.astype("int").tolist()

print(baseline)

In [ ]:
# cell 27
# check baseline prediction E[f(x)]
pred_baseline = predictor.predict(baseline)
print(pred_baseline)

### Clarify 구성 설정하기

Korean: 설정하기

다음으로, Clarify를 통한 설명 가능성 분석을 시작하기 위한 추가 구성을 설정하겠습니다. 다음 항목을 설정해야 합니다:
  1. **SHAPConfig**: 기준선을 생성합니다. 이 예제에서는 mean_abs가 모든 인스턴스에 대한 절대 SHAP 값의 평균으로, 기준선으로 지정됩니다.
  1. **DataConfig**: SageMaker Clarify에 데이터 I/O에 관한 기본 정보를 제공합니다. 입력 데이터셋의 위치, 출력 저장 위치, 헤더 이름 및 데이터셋 유형을 지정합니다.
  1. **ModelConfig**: 학습된 모델에 대한 정보를 지정합니다. 여기서는 앞서 생성한 모델 이름을 재사용합니다.
  
  참고: 프로덕션 모델에 대한 추가 트래픽을 방지하기 위해 SageMaker Clarify는 처리 중에 임시 엔드포인트를 설정하고 해제합니다. ModelConfig는 Clarify 처리 중에 모델을 실행하는 데 사용할 선호하는 인스턴스 유형과 인스턴스 수를 지정합니다.
  
Clarify에 대한 이러한 구성의 의미에 대해 자세히 알아보려면 다음 문서를 참조하세요: https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html

In [ ]:
# cell 28
shap_config = clarify.SHAPConfig(
    baseline=[baseline],
    num_samples=2000,  # num_samples are permutations from your features, so should be large enough as compared to number of input features, for example, 2k + 2* num_features
    agg_method="mean_abs",
    use_logit=True,
)  # we want the shap values to have log-odds units so that the equation 'shap values + expected probability =  predicted probability' for each instance record )

In [ ]:
# cell 29
explainability_report_prefix = "{}/clarify-explainability".format(prefix)
explainability_output_path = "s3://{}/{}".format(bucket, explainability_report_prefix)

explainability_data_config = clarify.DataConfig(
    s3_data_input_path=test_raw,
    s3_output_path=explainability_output_path,
    # label='credit_risk', # target column is not present in the test dataset
    headers=test_columns,
    dataset_type="text/csv",
)

In [ ]:
# cell 30
model_config = clarify.ModelConfig(
    model_name=pipeline_model.name,  # specify the inference pipeline model name
    instance_type="ml.c5.xlarge",
    instance_count=1,
    accept_type="text/csv",
)

### SageMaker Clarify 설명 가능성 작업 실행

모든 구성이 준비되었습니다. 이제 설명 가능성 작업을 시작해 보겠습니다. 이 작업은 임시 SageMaker 엔드포인트를 생성하고 해당 엔드포인트에서 추론을 수행하며 설명을 계산합니다. 기존의 프로덕션 엔드포인트 배포는 사용하지 않습니다.

#### 참고: 이 셀은 약 5-8분 동안 실행됩니다! 기다려 주세요.
SageMaker Clarify에 대한 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)에서 문서를 참조할 수 있습니다.

In [ ]:
# cell 31
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### 설명 가능성 보고서 보기

작업이 완료되면 Studio의 'Experiments and trials' 탭에서 설명 가능성 보고서를 볼 수 있습니다.

'clarify-explainability-'라는 이름의 트라이얼 컴포넌트를 찾아 설명 가능성 탭을 확인하세요.

아직 Studio 사용자가 아니라면, 다음 S3 버킷에서 이 보고서에 접근할 수 있습니다.

이 보고서는 입력 데이터셋을 사용한 모델에 대한 전역 설명을 포함하고 있습니다.

In [ ]:
# cell 32
explainability_output_path

In [ ]:
# cell 33
run_explainability_job_name = clarify_processor.latest_job.job_name
run_explainability_job_name


In [ ]:
# cell 34
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
# cell 35
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
# cell 36
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, explainability_report_prefix
        )
    )
)

In [ ]:
# cell 37
explainability_output_path

### S3에서 보고서 다운로드

In [ ]:
# cell 38
!aws s3 ls $explainability_output_path/


In [ ]:
# cell 39
!aws s3 cp --recursive $explainability_output_path ./explainability_report/

#### 아래의 설명 가능성 PDF 보고서를 통해 모델에 대한 SHAP를 사용한 전역 설명을 확인하세요. 이 보고서에는 데이터셋의 모든 개별 인스턴스에 대한 SHAP 요약 플롯도 포함되어 있습니다.

In [ ]:
# cell 40
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./explainability_report/report.html">Explainability Report</a></b>'))



### SageMaker Studio에서 설명 가능성 보고서 보기

Studio의 실험 탭에서 설명 가능성 보고서를 볼 수도 있습니다.

![title](explainability_detail.gif)

### Clarify를 통한 개별 예측의 로컬 설명 분석하기

#### 이 섹션의 선행 조건은 앞서 선택적 섹션에서 SageMaker 엔드포인트로 추론을 실행하여 테스트 데이터셋에 대한 개별 예측을 생성했어야 합니다
이 섹션에서는 Clarify가 생성한 각 개별 예측에 대한 로컬 설명 가능성 결과를 분석하고 이해하겠습니다. Clarify는 각 예측별로 각 특성에 대한 SHAP 값이 포함된 CSV 파일을 생성합니다. 이 CSV 파일을 다운로드해 보겠습니다.

In [ ]:
# cell 41
from sagemaker.s3 import S3Downloader
import json
import io

# read the shap values
S3Downloader.download(s3_uri=explainability_output_path + "/explanations_shap", local_path="output")
shap_values_df = pd.read_csv("output/out.csv")
print(shap_values_df.shape)

SHAP은 기본적으로 분류기 모델을 로지스틱 링크 함수 이전의 마진 출력 측면에서 설명합니다. 이는 SHAP 출력의 단위가 로그 오즈(log-odds) 단위이며, 음수 값은 0.5 미만의 확률을 의미하여 불량 신용 등급(클래스 0)을 나타냅니다.

#### 로지스틱 링크 함수 이전의 예측 출력 및 SHAP 값에 대한 간략한 기술 요약

y = f(x)는 예측 출력에 대한 로그 오즈(logit) 단위입니다.

E(y)는 입력 기준선에 대한 예측의 로그 오즈(logit) 단위입니다.

SHAP 값도 로그 오즈 단위로 표시됩니다.

모든 개별 예측에 대해 다음이 성립해야 합니다:

sum(SHAP values) + E(y)) == model_prediction_logit

logistic(model_prediction_logit) = model_prediction_probability

E(y) < 0은 기준 확률이 0.5 미만(불량 신용 기준선)임을 의미합니다.

E(y) > 0은 기준 확률이 0.5 초과(양호한 신용 기준선)임을 의미합니다.

y < 0은 예측 확률이 0.5 미만(불량 신용)임을 의미합니다.

y > 0은 예측 확률이 0.5 초과(양호한 신용)임을 의미합니다.

기준 입력에 대한 예측의 로그 오즈 단위인 E(y)를 검색할 수 있습니다.

In [ ]:
# cell 42
# get the base expected value to be used to plot SHAP values
S3Downloader.download(s3_uri=explainability_output_path + "/analysis.json", local_path="output")

with open("output/analysis.json") as json_file:
    data = json.load(json_file)
    base_value = data["explanations"]["kernel_shap"]["label0"]["expected_value"]

print("E(y): ", base_value)

앞서 설명한 대로, 불량 신용 예측을 대조하고 설명하기 위해 SHAP과 함께 사용할 양호한 신용 예측을 나타내는 기준선이 있습니다. E(y) > 0은 기준 확률이 0.5보다 크다는 것을 의미합니다(양호한 신용 기준선).

### 이전에 추론 중에 생성된 모델 예측을 포함하는 데이터프레임 생성

In [ ]:
# cell 43
from pandas import DataFrame

predictions_df = DataFrame(predictions, columns=["probability_score"])

predictions_df

### 예측, SHAP 값 및 테스트 데이터 결합

이제 모든 테스트 데이터 행과 해당 SHAP 값 및 예측 점수를 포함하는 단일 데이터프레임을 생성합니다.

In [ ]:
# cell 44
# join the probability score and shap values together in a single data frame
predictions_df.reset_index(drop=True, inplace=True)
shap_values_df.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

prediction_shap_df = pd.concat([predictions_df, shap_values_df, test_data], axis=1)
prediction_shap_df["probability_score"] = pd.to_numeric(
    prediction_shap_df["probability_score"], downcast="float"
)

prediction_shap_df

### 확률 점수를 이진 예측으로 변환

이제 확률 점수를 임계값(0.5)을 기준으로 이진 값(1/0)으로 변환합니다. 0.5보다 큰 확률 점수는 긍정적 결과(양호한 신용)이고 그보다 작은 점수는 부정적 결과(불량 신용)입니다.

In [ ]:
# cell 45
# create a new column as 'Prediction' converting the probability score to either 1 or 0
prediction_shap_df.insert(
    0, "Prediction", (prediction_shap_df["probability_score"] > 0.5).astype(int)
)

prediction_shap_df

### 불량 신용 예측만 필터링

이 사용 사례에서는 부정적 결과(불량 신용 예측)만 설명하는 데 관심이 있으므로, 예측이 0인 레코드만 유지하도록 필터링합니다.

In [ ]:
# cell 46
bad_credit_outcomes_df = prediction_shap_df[prediction_shap_df.iloc[:, 0] == 0]
bad_credit_outcomes_df

### SHAP 플롯 생성

이제 다양한 특성이 특정 부정적 결과에 얼마나 기여했는지 이해하기 위해 추가 SHAP 플롯을 생성해 보겠습니다.

#### 더 많은 시각화를 위한 오픈 소스 SHAP 라이브러리 설치

In [ ]:
# cell 47
!conda install -c conda-forge shap -y

In [ ]:
# cell 48
import shap

#### 단일 불량 신용 앙상블 예측 인스턴스에 대한 SHAP 설명 플롯. 가장 낮은 확률을 가진 예측 인스턴스를 선택하겠습니다.

In [ ]:
# cell 49
import matplotlib.pyplot as plt

min_index = prediction_shap_df["probability_score"].idxmin()
print(min_index)
print("mean probability of dataset")
print(prediction_shap_df[["probability_score"]].mean())
print("individual probability")
print(prediction_shap_df.iloc[min_index, 1])
print("sum of shap values")
print(prediction_shap_df.iloc[min_index, 2:22].sum())
print("base value from analysis.json")
print(base_value)

'불량 신용' 예측 SHAP 값 예시.

아래 차트에서 f(x)는 이 특정 개별 인스턴스의 예측을 로그 오즈 단위로 나타냅니다. 음수인 경우 불량 신용 예측을 의미합니다.

아래 차트에서 E(f(x))는 기준선 입력의 예측을 로그 오즈 단위로 나타냅니다. 양수이므로 양호한 신용 클래스에 속합니다.

개별 예시는 양호한 신용 기준선과 대조됩니다. 따라서 음수 SHAP 값을 가진 특성들이 초기 기준선 양수 값에서 최종 부정적 결정으로 이끕니다.

### 아래 예시에서 입력 특성 (status = 1), (purpose = 0) 및 (personal_status_sex = 2)는 부정적 결정을 이끄는 상위 3개 특성입니다.

이러한 값들이 논리적 카테고리에 어떻게 매핑되는지 이해하려면 데이터 설명을 참조할 수 있습니다.

In [ ]:
# cell 50
import inspect
import shap.plots._waterfall
source = inspect.getsource(shap.plots._waterfall)
new_source = source.replace("as pl", "as plt")
exec(new_source, shap.plots._waterfall.__dict__)

explanation_obj = shap._explanation.Explanation(
    values=prediction_shap_df.iloc[min_index, 2:22].to_numpy(),
    base_values=base_value,
    data=test_data.iloc[min_index].to_numpy(),
    feature_names=test_data.columns,
)
shap.plots.waterfall(shap_values=explanation_obj, max_display=20, show=False)

위의 플롯에서 min_index를 변경하여 다른 개별 인스턴스의 예측을 설명해 볼 수 있습니다.

### Amazon SageMaker Clarify로 데이터 편향 감지
#### Amazon Science: [Clarify가 머신 러닝 개발자가 의도하지 않은 편향을 감지하는 데 어떻게 도움이 되는지](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

#### [편향 및 공정성에 대한 Clarify 용어](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-detect-data-bias.html)

#### [사전 훈련 편향 메트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html)

#### [사후 훈련 편향 메트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html)

#### 사전 훈련 및 사후 훈련 편향 메트릭 계산

참고: 사전 훈련 및 사후 훈련 편향 감지 작업을 별도로 실행할 수도 있습니다.

DataConfig 객체는 Clarify에 데이터 I/O에 대한 기본 정보를 전달합니다. 입력 데이터셋의 위치, 출력을 저장할 위치, 대상 열(레이블), 헤더 이름 및 데이터셋 유형을 지정합니다.

마찬가지로, ModelConfig(이전에 설명 가능성 작업을 위해 생성됨) 객체는 훈련된 모델에 대한 정보를 전달하고 ModelPredictedLabelConfig는 예측 형식에 대한 정보를 제공합니다.

In [ ]:
# cell 51
bias_report_prefix = "{}/clarify-bias".format( prefix)
bias_report_output_path = "s3://{}/{}".format(bucket,bias_report_prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_raw,
    s3_output_path=bias_report_output_path,
    label="credit_risk",
    headers=training_data.columns.to_list(),
    dataset_type="text/csv",
)
predictions_config = clarify.ModelPredictedLabelConfig(label=None, probability=0)

SageMaker Clarify는 민감한 열(facets)과 바람직한 결과(facet_values_or_threshold)도 필요로 합니다.

BiasConfig API에서 이 정보를 지정합니다. 여기서 age는 분석하는 facet이고 40은 임계값입니다. 'personal_status_sex' 그룹은 조건부 인구통계 격차(CDD) 메트릭 측정을 위한 하위 그룹을 형성하는 데 사용됩니다.

In [ ]:
# cell 52
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1],
    facet_name="age",
    facet_values_or_threshold=[40],
    group_name="personal_status_sex",
)

#### 참고: 이 셀은 약 5-8분 동안 실행됩니다! 기다려 주세요.
SageMaker Clarify에 대한 자세한 문서는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)에서 참조할 수 있습니다.

In [ ]:
# cell 53
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

#### 편향 감지 보고서 보기
Studio의 실험 탭에서 편향 감지 보고서를 볼 수 있습니다.

아직 Studio 사용자가 아닌 경우, 다음 S3 버킷에서 이 보고서에 접근할 수 있습니다.

In [ ]:
# cell 54
bias_report_output_path

In [ ]:
# cell 55
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

In [ ]:
# cell 56

from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)



In [ ]:
# cell 57
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
# cell 58
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

#### S3에서 보고서 다운로드

In [ ]:
# cell 59
!aws s3 ls $bias_report_output_path/

In [ ]:
# cell 60
bias_report_output_path

In [ ]:
# cell 61
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

#### 사전 훈련 편향 및 사후 훈련 편향 지표가 포함된 편향 보고서 PDF를 확인합니다.

In [ ]:
# cell 62
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

### Studio에서 편향 보고서 보기

또는 Studio의 실험 탭에서 편향 보고서를 볼 수도 있습니다. 각 편향 지표에는 탐색할 수 있는 자세한 설명과 예시가 포함되어 있습니다. 결과를 유용한 표로 요약할 수도 있습니다!

![title](bias_report.gif)

몇 가지 사전 훈련 및 사후 훈련 편향 지표를 구체적으로 살펴보겠습니다.

사전 훈련 편향 지표
1. 클래스 불균형(Class imbalance)
2. DPL - 실제 레이블의 긍정적 비율 차이

사후 훈련 편향 지표
1. DPPL - 예측 레이블의 긍정적 비율 차이
2. DI - 불균형 영향

In [ ]:
# cell 63
S3Downloader.download(s3_uri=bias_report_output_path + "/analysis.json", local_path="output")

with open("output/analysis.json") as json_file:
    data = json.load(json_file)
    print("pre-training bias metrics")
    class_imbalance = data["pre_training_bias_metrics"]["facets"]["age"][0]["metrics"][1]["value"]
    print("class imbalance: ", class_imbalance)
    DPL = data["pre_training_bias_metrics"]["facets"]["age"][0]["metrics"][2]["value"]
    print("DPL: ", DPL)
    print("\n")
    print("post training bias metrics")
    DPPL = data["post_training_bias_metrics"]["facets"]["age"][0]["metrics"][6]["value"]
    print("DPPL: ", DPPL)
    DI = data["post_training_bias_metrics"]["facets"]["age"][0]["metrics"][5]["value"]
    print("DI: ", DI)

여기서 임계값이 [40]인 "age" 측면에 대해 [CI](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-bias-metric-class-imbalance.html)와 [DI](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-di.html)는 높은 반면, [DPL](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-data-bias-metric-true-label-imbalance.html)과 [DPPL](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-dppl.html)은 낮은 것을 볼 수 있습니다. 사전 훈련 편향을 완화하기 위해 데이터 전처리 기술을 적용할 수 있으며, 사후 훈련 편향을 완화하기 위해 훈련 알고리즘을 재평가할 수 있습니다.

### 6. 정리
마지막으로, 이 데모를 위해 설정하고 사용한 리소스를 정리하는 것을 잊지 마세요!

In [ ]:
# cell 64
session.delete_endpoint(endpoint_name)

In [ ]:
# cell 65
session.delete_model(pipeline_model.name)

## 7. 추가 탐색 자료

* [더 공정한 머신 러닝을 향해](https://www.amazon.science/research-awards/success-stories/algorithmic-bias-and-fairness-in-machine-learning)
* [금융 분야 머신 러닝을 위한 공정성 측정](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance.pdf)
* [Amazon SageMaker Clarify: 클라우드에서의 머신 러닝 편향 감지 및 설명 가능성](https://www.amazon.science/publications/amazon-sagemaker-clarify-machine-learning-bias-detection-and-explainability-in-the-cloud)
* [Amazon AI 공정성 및 설명 가능성 백서](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)
* [Clarify가 머신 러닝 개발자가 의도하지 않은 편향을 감지하는 데 어떻게 도움이 되는지](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)